# Объединение датасетов (СберЗдоровье, Prodoctorov)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
sber = pd.read_csv('https://github.com/yyaroslavskiy/cuddly-chainsaw/raw/refs/heads/develop/eda/preparation-to-merge/sber_clean.csv', index_col = 0)
sberr = pd.read_csv('https://github.com/yyaroslavskiy/cuddly-chainsaw/raw/refs/heads/develop/eda/preparation-to-merge/sber_reviews_clean.csv', index_col = 0)

prod = pd.read_csv('https://github.com/yyaroslavskiy/cuddly-chainsaw/raw/refs/heads/develop/eda/preparation-to-merge/prodoctorov_clean.csv', index_col = 0)
prodr = pd.read_csv('https://github.com/yyaroslavskiy/cuddly-chainsaw/raw/refs/heads/develop/eda/preparation-to-merge/prodoctorov_reviews_clean.csv', index_col = 0)

## Merge

Приведём специальности и стаж к одному типу значений

In [ ]:
prod['speciality'] = prod['speciality'].apply(lambda x: [i.strip() for i in x.split(',')])
sber['experience'] = sber['experience'].astype('Int64')
prod['experience'] = prod['experience'].astype('Int64')

Разделим на полных тёзок и нетёзок. Тёзок соединим по имени и стажу, нетёзок по ФИО

In [ ]:
sber_unique = sber.drop_duplicates(subset = ['name'], keep = False)
sber_dups = sber[sber.duplicated(subset = ['name'], keep = False)]

prod_unique = prod.drop_duplicates(subset = ['name'], keep = False)
prod_dups = prod[prod.duplicated(subset = ['name'], keep = False)]

In [ ]:
merged_unique = pd.merge(how = 'outer', left = sber_unique, right = prod_unique, on = 'name', suffixes = ('_sber', '_prod'))
merged_unique

Внесём параметр дельта, чтобы проверить на выбросы

In [ ]:
merged_unique['delta'] = np.where(
    merged_unique['experience_sber'].isna() | merged_unique['experience_prod'].isna(),
    np.nan,
    np.abs(merged_unique['experience_sber'] - merged_unique['experience_prod'])
)

Если разница в стажах больше 10 лет, то это уже непорядок

In [ ]:
merged_unique['delta'].value_counts()

In [ ]:
merged_unique = merged_unique[(merged_unique['delta'] <= 10) | merged_unique['delta'].isna()]

Объединим показатель стажа по максимуму из колонок и удалим дельту

In [ ]:
merged_unique = merged_unique.copy()
merged_unique['experience'] = np.fmax(merged_unique['experience_sber'], merged_unique['experience_prod'])
merged_unique = merged_unique.drop(['experience_sber', 'experience_prod', 'delta'], axis=1)

In [ ]:
merged_unique

Теперь соединим тёзок

In [ ]:
merged_dups = pd.merge(how = 'outer', left = sber_dups, right = prod_dups, on = ['name', 'experience'], suffixes = ('_sber', '_prod'))
merged_dups

Слепим общий датасет

In [ ]:
merged_total = pd.concat([merged_dups, merged_unique])
merged_total

In [ ]:
merged_total.columns

## Feature Combination

Объединим специальности в единый массив

In [ ]:
def combine_specialties(row):
    s1 = row['speciality_sber'] if isinstance(row['speciality_sber'], list) else []
    s2 = row['speciality_prod'] if isinstance(row['speciality_prod'], list) else []

    s1 = list(map(str.lower, s1))
    s2 = list(map(str.lower, s2))

    set1 = set(s1)
    set2 = set(s2)
    combined_set = set1.union(set2)

    if combined_set:
        return list(combined_set)
    else:
        return pd.NA

merged_total['speciality'] = merged_total.apply(combine_specialties, axis = 1)

In [ ]:
merged_total['speciality']

In [ ]:
merged_total = merged_total.drop(['speciality_sber', 'speciality_prod'], axis=1)

In [ ]:
merged_total.columns

И is_kids, is_adults тоже объединим в один параметр по правилам объединения множеств

In [ ]:
def combine_bool_kids(row):
    if isinstance(row['is_kids_sber'], bool) and isinstance(row['is_kids_prod'], bool):
      return row['is_kids_sber'] or row['is_kids_prod']
    elif isinstance(row['is_kids_sber'], bool):
      return row['is_kids_sber']
    elif isinstance(row['is_kids_prod'], bool):
      return row['is_kids_prod']
    else:
      return pd.NA

def combine_bool_adults(row):
    if isinstance(row['is_adults_sber'], bool) and isinstance(row['is_adults_prod'], bool):
      return row['is_adults_sber'] or row['is_adults_prod']
    elif isinstance(row['is_adults_sber'], bool):
      return row['is_adults_sber']
    elif isinstance(row['is_adults_prod'], bool):
      return row['is_adults_prod']
    else:
      return pd.NA

merged_total['is_kids'] = merged_total.apply(combine_bool_kids, axis = 1)
merged_total['is_adults'] = merged_total.apply(combine_bool_adults, axis = 1)

In [ ]:
merged_total = merged_total.drop(['is_kids_sber', 'is_adults_sber', 'is_kids_prod', 'is_adults_prod'], axis=1)

In [ ]:
merged_total.columns

## Сохранение в csv

Красота! Теперь можно и в csv сохранить.

In [ ]:
merged_total.to_csv('doctors.csv')

Отзывы объединим и тоже сохраним в общий csv-файл

In [ ]:
pd.concat([prodr, sberr]).to_csv('doctors_review.csv')